# Rotation of rasters and subsequent processing

In this notebook I am going to attempt to rotate a raster, write it to a file, perform an analysis on the rotated raster, and then rotate it back.

In [ ]:
import cartopy as cp
import cartopy.crs as ccrs
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
DEMdata = rio.open('lg_conception_SRTM30_UTM.bil')
array = DEMdata.read(1)
type(array)

In [ ]:
from scipy import ndimage, misc
import matplotlib.pyplot as plt

In [ ]:
array_20 = ndimage.rotate(array, 20)
array_40 = ndimage.rotate(array, 40)

In [ ]:
plt.imshow(array, cmap='gray')

In [ ]:
plt.imshow(array_20, cmap='gray')

In [ ]:
plt.imshow(array_40, cmap='gray')

### Lets get rid of the annoying nodata

In [ ]:
array_40[array_40<=0]=-9999

# Here is one that has ndv values for plotting
array_40_ndv = np.copy(array_40)
array_40_ndv[array_40_ndv<=0]=np.nan

In [ ]:
plt.imshow(array_40, cmap='gray')

In [ ]:
plt.imshow(array_40_ndv, cmap='gray')

## Now for the trickier part. We need to give an imposter georeferencing

In [ ]:
So the array is rotated. We now need to give this raster a georeferencing 

## Now lets manipulate these rasters using lsdtopotools

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw

### A function for rotating and mapping rasters

In [ ]:
def array2raster_rotated(rasterfn,newRasterfn,driver_name = "ENVI", noDataValue = -9999,rotation=0):
    """Takes an array and writes to a GDAL compatible raster. It needs another raster to map the dimensions.
    Args:
        FileName (str): The filename (with path and extension) of a raster that has the same dimensions as the raster to be written.
        newRasterfn (str): The filename (with path and extension) of the new raster.
        driver_name (str): The type of raster to write. Default is ENVI since that is the LSDTOpoTools format
        noDataValue (float): The no data value
    Return:
        np.array: A numpy array with the data from the raster.
    Author: SMM
    """
    import osgeo.gdal as gdal
    import osgeo.gdal_array as gdal_array
    from osgeo import osr
    import numpy as np   
    from scipy import ndimage, misc

    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    cols = raster.RasterXSize
    rows = raster.RasterYSize
    
    YMax = originY+pixelHeight*rows
    
    print("The original raster details are:")
    print("xll: "+str(originX))
    print("yll: "+str(originY))
    print("ymax: "+str(YMax))
    print("dx: " +str(pixelWidth))
    print("dy: " +str(pixelHeight))
    print("cols: " +str(cols))
    print("rows: " +str(cols))
    
    # Now we rotate the raster
    raster_array = np.array(raster.GetRasterBand(1).ReadAsArray())
    
    # We can't deal with -9999 or nan, so we use small negative numebers to pad the DEM
    raster_array[raster_array<=0]=-2
    rotated_array = ndimage.rotate(raster_array, rotation)
    #rotated_array = np.nan_to_num(rotated_array,nan=-9999)
    rotated_array[rotated_array<=0]=-9999
    
    # get the shape of this raster:
    raster_dim = rotated_array.shape
    print("Raster dimensions are:")
    print(raster_dim)
    new_rows = raster_dim[0]
    new_cols = raster_dim[1]
    
    # We use the same corners
    new_originX = originX
    new_originY = originY
    

    driver = gdal.GetDriverByName(driver_name)
    outRaster = driver.Create(newRasterfn, new_cols, new_rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((new_originX, pixelWidth, 0, new_originY, 0, pixelHeight))
    outRaster.GetRasterBand(1).SetNoDataValue( noDataValue )
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(rotated_array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromWkt(raster.GetProjectionRef())
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    raster=None
    outRaster=None

    # Get the raster prefix
    SplitRasterfile = newRasterfn.split(".")
    RasterPrefix = ".".join(SplitRasterfile[:-1])
    hdrname = RasterPrefix+".hdr"
    print("The raster prefix is: "+RasterPrefix)

    if driver_name == "ENVI":

        with open(hdrname,"a") as f:
            #print("Appending data div to "+hdrname)
            f.write('data ignore value = '+str(noDataValue)+"\n")

In [ ]:
raster_in_fname = 'lg_conception_SRTM30_UTM.bil'
raster_out_fname = 'lg_rotate9.bil'
array2raster_rotated(raster_in_fname,raster_out_fname,rotation=40)

In [ ]:
!ls

## Lets look at the resulting rasters

In [ ]:
import cartopy as cp
import cartopy.crs as ccrs

In [ ]:
DEMdata = rio.open('lg_conception_SRTM30_UTM.bil')
bounds = DEMdata.bounds
Extent = [bounds.left,bounds.right,bounds.bottom,bounds.top]
array = DEMdata.read(1)
DEMdata.crs

In [ ]:
print(Extent)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.UTM(10))
# Limit the extent of the map to a small longitude/latitude range.
ax.set_extent(Extent, crs=ccrs.UTM(10))
#ax.coastlines()
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
#print('Projecting and plotting image (this may take a while)...')
ax.imshow(array, extent=Extent, transform=ccrs.UTM(10), origin="upper", cmap='terrain',alpha=0.5,zorder=2)
fig.savefig("test.png", dpi=300)

now for the rotated

In [ ]:
DEMdata2 = rio.open('lg_rotate3.bil')
bounds = DEMdata2.bounds
Extent2 = [bounds.left,bounds.right,bounds.bottom,bounds.top]
array2 = DEMdata2.read(1)
DEMdata2.crs

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.UTM(10))
# Limit the extent of the map to a small longitude/latitude range.
ax.set_extent(Extent2, crs=ccrs.UTM(10))
#ax.coastlines()
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
#print('Projecting and plotting image (this may take a while)...')
ax.imshow(array2, extent=Extent, transform=ccrs.UTM(10), origin="upper", cmap='terrain',alpha=0.5,zorder=2)
fig.savefig("test.png", dpi=300)

## Now lets get the drainage extraction and junction angles

In [ ]:
Dataset_prefix="lg_rotate9" 
lsdtt_parameters = {"write_hillshade" : "true",
                    "print_junction_angles_to_csv" : "true"}
r_prefix = Dataset_prefix
w_prefix = Dataset_prefix
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

## Rotate back into the original coordinates

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
df = pd.read_csv("lg_rotate9_FULL_JAngles.csv")

In [ ]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf = gdf.set_crs(epsg=4326)
gdf.head()

Now we get the locations in UTM (not these are on the rotated raster)

In [ ]:
gdf2 = gdf.to_crs("EPSG:32610")
gdf2.head()

In [ ]:
geo = gdf2.geometry
print(geo)

In [ ]:
gdf2["geometry"].apply(lambda p: p.x.rename(columns="x"))

In [ ]:
gdf2['x'] = gdf2['geometry'].apply(lambda p: p.x)
gdf2['y'] = gdf2['geometry'].apply(lambda p: p.y)
gdf2.head()